In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.4 MB/s eta 0:00:00


In [ ]:
!pip -q uninstall -y peft bitsandbytes albumentations albucore opencv-python opencv-python-headless || true


!pip -q install "transformers==4.41.2" "decord==0.6.0" "torchmetrics==1.4.0" "scikit-learn==1.5.1" \
                albumentations==1.4.8 albucore==0.0.14 opencv-python-headless==4.10.0.84 --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 116.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 185.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 338.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 129.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 315.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 234.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.1 which is incompatible.


In [ ]:
# CONFIG

from pathlib import Path

# Input / Output
VIDEO_PATH           = "/content/cows.mp4"
OUT_DIR              = Path("/content/cow_logs")
WRITE_ANNOTATED_MP4  = True

# YOLO / ByteTrack
YOLO_WEIGHTS         = "/content/best.pt"
YOLO_IMGSZ           = 960
YOLO_CONF            = 0.60
YOLO_IOU             = 0.50
TRACKER_YAML         = "bytetrack.yaml"

# Two head TimeSformer shared backbone + 2 heads classifiers
MULTI_MODEL_DIR      = "/content/drive/MyDrive/Models/timesformer-cows-multitask/backbone_hf"
CLF_RES              = 224               # 112 for speed
WINDOW_FRAMES        = 12                # 12 frames/clip
SAMPLE_FPS           = 1.6               # 16 frames per 10 s
WINDOW_OVERLAP       = 0.5               # 0.5, decision every 5 s at 1.6 fps
SMOOTH_K             = 3                 # majority over last K labels
MIN_WARMUP_FRAMES    = 8

# Segmentation / tracking timeouts
INACTIVITY_TIMEOUT_S = 10
MIN_SEGMENT_S        = 2

# Final joint 7-class space
CLASSES = [
    "Standing",
    "Lying",
    "Drinking",
    "Feeding & Standing",
    "Feeding & Lying",
    "Ruminating & Standing",
    "Ruminating & Lying",
]
LABEL2ID = {c:i for i,c in enumerate(CLASSES)}
ID2LABEL = {i:c for c,i in LABEL2ID.items()}

# Heads internal label spaces
POSTURE  = {"Standing": 0, "Lying": 1}
ACTIVITY = {"None": 0, "Drinking": 1, "Feeding": 2, "Ruminating": 3}

# Valid (posture, activity) combinations, final 7-class index
PAIR_LIST = [
    ("Standing", "None",        LABEL2ID["Standing"]),
    ("Lying",    "None",        LABEL2ID["Lying"]),
    ("Standing", "Drinking",    LABEL2ID["Drinking"]),
    ("Standing", "Feeding",     LABEL2ID["Feeding & Standing"]),
    ("Lying",    "Feeding",     LABEL2ID["Feeding & Lying"]),
    ("Standing", "Ruminating",  LABEL2ID["Ruminating & Standing"]),
    ("Lying",    "Ruminating",  LABEL2ID["Ruminating & Lying"]),
]

PAD_TARGET_HW = (640, 640)

OUT_DIR.mkdir(parents=True, exist_ok=True)
print("Configured (two-head).")


Configured (two-head).


In [ ]:
# 2) Imports & helpers

import os, time, math, json, gc
import cv2, torch, numpy as np, pandas as pd
from collections import defaultdict, deque
from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F

from ultralytics import YOLO
from transformers import AutoConfig, TimesformerModel

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

def sec_to_hms(t):
    h = int(t//3600); m = int((t%3600)//60); s = int(t%60)
    return f"{h:02d}:{m:02d}:{s:02d}"

# Scale to fit center padding
def center_pad_bbox_crop(img_rgb, xyxy, pad_target=(640, 640), out_size=224):
    """
    Crop YOLO bbox, if larger than pad canvas, scale down to fit, center-pad to pad_target
    ,resize to out_size. Returns RGB uint8 or None if bbox empty.
    """
    H, W = img_rgb.shape[:2]
    x1, y1, x2, y2 = map(int, xyxy)
    x1 = max(0, x1); y1 = max(0, y1); x2 = min(W-1, x2); y2 = min(H-1, y2)
    crop = img_rgb[y1:y2, x1:x2]
    if crop.size == 0:
        return None

    th, tw = pad_target
    h, w = crop.shape[:2]
    if h > th or w > tw:
        scale = min(th / h, tw / w)
        nh, nw = max(1, int(round(h * scale))), max(1, int(round(w * scale)))
        crop = cv2.resize(crop, (nw, nh), interpolation=cv2.INTER_LINEAR)
        h, w = nh, nw

    top = (th - h) // 2; bottom = th - h - top
    left = (tw - w) // 2; right = tw - w - left
    padded = cv2.copyMakeBorder(crop, top, bottom, left, right,
                                borderType=cv2.BORDER_CONSTANT, value=(0, 0, 0))

    if out_size and (out_size != th):
        padded = cv2.resize(padded, (out_size, out_size), interpolation=cv2.INTER_LINEAR)
    return padded

# ImageNet norm
IMNET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMNET_STD  = np.array([0.229, 0.224, 0.225], dtype=np.float32)
def preprocess_clip(frames_hw3):  # list of T RGB (H,W,3) uint8
    arr = np.stack(frames_hw3, axis=0).astype(np.float32) / 255.0
    arr = (arr - IMNET_MEAN) / IMNET_STD
    arr = arr.transpose(0,3,1,2)  # T,C,H,W
    return torch.from_numpy(arr).unsqueeze(0)  # 1,T,C,H,W

def majority(lst):
    if not lst: return None
    return max(set(lst), key=lst.count)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Device: cuda


In [ ]:
# Two-head TimeSformer backbone + heads + joint decode

class MultiTaskTimeSformer(nn.Module):
    """
    Loads a Timesformer backbone from HF ckpt and adds two linear heads:
      - posture (Standing/Lying), 2 classes
      - activity (None/Drinking/Feeding/Ruminating), 4 classes
    """
    def __init__(self, ckpt_dir: str):
        super().__init__()
        cfg = AutoConfig.from_pretrained(ckpt_dir)
        self.backbone = TimesformerModel.from_pretrained(ckpt_dir, config=cfg)
        hidden = cfg.hidden_size
        p_drop = getattr(cfg, "hidden_dropout_prob", getattr(cfg, "dropout", 0.0))
        self.dropout = nn.Dropout(p_drop)
        self.posture_head  = nn.Linear(hidden, 2)
        self.activity_head = nn.Linear(hidden, 4)

    def forward(self, pixel_values):
        # pixel_values: [B, T, C, H, W]  (as constructed in preprocess_clip)
        out = self.backbone(pixel_values=pixel_values)       # Base model forward
        cls = out.last_hidden_state[:, 0]                    # CLS token
        x = self.dropout(cls)
        return self.posture_head(x), self.activity_head(x)   # logits_p, logits_a

@torch.no_grad()
def joint_decode(posture_logits, activity_logits):
    """
    Combine posture+activity heads into the 7 class label space via masked log sum.
    Returns: pred_idx [B], joint_log_scores [B, 7]
    """
    lp = F.log_softmax(posture_logits,  dim=-1)   # [B,2]
    la = F.log_softmax(activity_logits, dim=-1)   # [B,4]
    B = lp.size(0)
    joint = torch.full((B, len(CLASSES)), -1e9, device=lp.device)

    for p_str, a_str, cls_idx in PAIR_LIST:
        p = POSTURE[p_str]; a = ACTIVITY[a_str]
        joint[:, cls_idx] = lp[:, p] + la[:, a]   # log-space sum == add

    preds = joint.argmax(-1)
    return preds, joint

# Instantiate two-head model
mt_cfg = AutoConfig.from_pretrained(MULTI_MODEL_DIR)  # for info/consistency
clf_twohead = MultiTaskTimeSformer(MULTI_MODEL_DIR).to(device).eval()
print("Two-head TimeSformer loaded.")


Two-head TimeSformer loaded.


In [ ]:
# Load YOLO detector + ByteTrack

yolo = YOLO(YOLO_WEIGHTS)
try:
    if device == "cuda":
        #try: yolo.model.half()
        pass
    yolo.fuse()
except Exception as e:
    print("YOLO optimize hint:", e)
print("YOLO loaded.")


YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs
YOLO loaded.


In [ ]:
# Main pipeline

# Video props
cap = cv2.VideoCapture(VIDEO_PATH)
assert cap.isOpened(), f"Cannot open video: {VIDEO_PATH}"
video_fps   = cap.get(cv2.CAP_PROP_FPS) or 25.0
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
video_w     = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_h     = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

# Sampling & windowing
sample_stride   = max(1, int(round(video_fps / max(0.1, float(SAMPLE_FPS)))))
step_frames     = max(1, int(round(WINDOW_FRAMES * (1.0 - float(WINDOW_OVERLAP)))))  # in SAMPLES
timeout_frames  = int(round(INACTIVITY_TIMEOUT_S * video_fps))

print(f"fps={video_fps:.2f} frames={frame_count} size=({video_w}x{video_h})")
print(f"Sample every {sample_stride} frames (~{SAMPLE_FPS} fps); window={WINDOW_FRAMES}, step={step_frames}, clf_res={CLF_RES}")

# Per-track state
buffers              = defaultdict(lambda: deque(maxlen=WINDOW_FRAMES))  # cow_id -> deque of padded crops
last_sample_frame    = defaultdict(lambda: -10**9)                        # cow_id -> last raw frame idx sampled
last_classify_frame  = defaultdict(lambda: -10**9)                        # cow_id -> last raw frame idx classified
last_seen_frame      = dict()                                            # cow_id -> last frame seen
pred_hist_joint      = defaultdict(lambda: deque(maxlen=SMOOTH_K))       # cow_id -> last K *final* labels (strings)

active_event = dict()  # cow_id -> {"label": str, "start_frame": int}
events       = []      # finalized segments

# Annotated video writer
writer = None
if WRITE_ANNOTATED_MP4:
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(str(OUT_DIR / "annotated.mp4"), fourcc, video_fps, (video_w, video_h))

start_time = time.time()
frame_idx  = -1

# ByteTrack stream
gen = yolo.track(
    source=VIDEO_PATH, stream=True, imgsz=YOLO_IMGSZ,
    conf=YOLO_CONF, iou=YOLO_IOU, tracker=TRACKER_YAML,
    device=0 if device=="cuda" else 'cpu', verbose=False, persist=True
)

# Ensure fast tracker dependency is present
try:
    import lap
except Exception:
    pass

DEBUG_DUMPS = 0  # set to 0 to save a few clip grids
_dbg_dumped = 0

for res in tqdm(gen, desc="Processing"):
    frame_idx += 1
    img_bgr = res.orig_img
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    xyxy = None; ids_arr = None
    if res.boxes is not None and res.boxes.xyxy is not None:
        xyxy = res.boxes.xyxy.cpu().numpy()
        ids  = res.boxes.id
        ids_arr = ids.cpu().numpy().astype(int) if ids is not None else np.arange(len(xyxy), dtype=int)

        for bb, tid in zip(xyxy, ids_arr):
            last_seen_frame[tid] = frame_idx

            # sample into clip buffer at SAMPLE_FPS
            if frame_idx - last_sample_frame[tid] >= sample_stride:
                crop = center_pad_bbox_crop(img_rgb, bb, pad_target=PAD_TARGET_HW, out_size=CLF_RES)
                if crop is not None:
                    buffers[tid].append(crop)
                    last_sample_frame[tid] = frame_idx

            # readiness, spaced by SAMPLES since last classify
            n_samples = len(buffers[tid])
            ready_full = (n_samples >= WINDOW_FRAMES)
            ready_warm = (active_event.get(tid) is None) and (n_samples >= MIN_WARMUP_FRAMES)
            step_ok    = (frame_idx - last_classify_frame[tid]) >= (sample_stride * step_frames)

            if (ready_full or ready_warm) and step_ok:
                clip = list(buffers[tid])
                if n_samples < WINDOW_FRAMES:
                    clip = clip + [clip[-1]] * (WINDOW_FRAMES - n_samples)  # pad to T

                pixel_values = preprocess_clip(clip).to(device)
                with torch.no_grad():
                    logits_p, logits_a = clf_twohead(pixel_values=pixel_values)   # two heads
                    pred_ids, joint = joint_decode(logits_p, logits_a)            # [B], [B,7]
                    pred_idx = int(pred_ids.item())
                    label    = ID2LABEL[pred_idx]

                pred_hist_joint[tid].append(label)
                smooth_label = majority(list(pred_hist_joint[tid]))
                last_classify_frame[tid] = frame_idx

                # Debug dump of exactly what the model saw
                if _dbg_dumped < DEBUG_DUMPS:
                    T = len(clip); cols=4; rows=math.ceil(T/cols)
                    padf = np.zeros_like(clip[0], dtype=np.uint8)
                    tiles = clip + [padf]*(rows*cols - T)
                    grid = np.concatenate([np.concatenate(tiles[r*cols:(r+1)*cols], 1) for r in range(rows)], 0)
                    out_path = OUT_DIR / f"dbg_cow{tid}_f{frame_idx}_raw-{label.replace(' ','_')}_sm-{smooth_label.replace(' ','_')}.jpg"
                    cv2.imwrite(str(out_path), cv2.cvtColor(grid, cv2.COLOR_RGB2BGR))
                    _dbg_dumped += 1

                # Segment around window center
                window_center_frame = frame_idx - (WINDOW_FRAMES // 2)
                if tid not in active_event:
                    active_event[tid] = {"label": smooth_label, "start_frame": max(0, window_center_frame)}
                else:
                    if smooth_label != active_event[tid]["label"]:
                        st = active_event[tid]["start_frame"]
                        et = max(st, window_center_frame)
                        if (et - st) / video_fps >= MIN_SEGMENT_S:
                            events.append({
                                "cow_id": int(tid),
                                "activity": active_event[tid]["label"],
                                "start_frame": st,
                                "end_frame": et
                            })
                        active_event[tid] = {"label": smooth_label, "start_frame": window_center_frame}

    # close stale tracks
    to_close = []
    for tid, lastf in list(last_seen_frame.items()):
        if frame_idx - lastf >= timeout_frames:
            to_close.append(tid)
    for tid in to_close:
        if tid in active_event:
            st = active_event[tid]["start_frame"]
            et = last_seen_frame[tid]
            if et < st: et = st
            if (et - st) / video_fps >= MIN_SEGMENT_S:
                events.append({
                    "cow_id": int(tid),
                    "activity": active_event[tid]["label"],
                    "start_frame": st,
                    "end_frame": et
                })
        buffers.pop(tid, None)
        pred_hist_joint.pop(tid, None)
        active_event.pop(tid, None)
        last_seen_frame.pop(tid, None)

    # overlay
    if WRITE_ANNOTATED_MP4 and (xyxy is not None):
        for bb, tid in zip(xyxy, ids_arr):
            lab = active_event.get(int(tid), {}).get("label")
            if lab is None:
                lab = "estimating…"
            x1,y1,x2,y2 = map(int, bb)
            cv2.rectangle(img_bgr, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.putText(img_bgr, f"{int(tid)}: {lab}", (x1, max(0,y1-6)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)
        writer.write(img_bgr)

# flush at end
for tid, ev in list(active_event.items()):
    st = ev["start_frame"]
    et = frame_idx
    if et < st: et = st
    if (et - st) / video_fps >= MIN_SEGMENT_S:
        events.append({
            "cow_id": int(tid),
            "activity": ev["label"],
            "start_frame": st,
            "end_frame": et
        })
if writer is not None:
    writer.release()

elapsed = time.time() - start_time
print(f"Processed {frame_idx+1} frames in {elapsed:.1f}s "
      f"(~{(frame_idx+1)/max(1.0,elapsed):.1f} FPS incl. det+track+class).")
if CLF_RES == 112:
    print("Note: 112×112 mode chosen (faster, slight accuracy drop).")




fps=30.00 frames=5399 size=(1920x1080)
Sample every 19 frames (~1.6 fps); window=12, step=6, clf_res=224
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect



Processing: 5399it [03:58, 22.63it/s]

Processed 5399 frames in 239.5s (~22.5 FPS incl. det+track+class).


In [ ]:
# Logs (CSV) + summary

def save_and_summarize(events, out_dir: Path, fps: float):
    if not events:
        print("No segments produced — check model paths / labels / thresholds.")
        return None, None

    df = pd.DataFrame(events)
    df["start_sec"]   = df["start_frame"] / fps
    df["end_sec"]     = df["end_frame"] / fps
    df["duration_s"]  = df["end_sec"] - df["start_sec"]
    df["start_hms"]   = df["start_sec"].map(sec_to_hms)
    df["end_hms"]     = df["end_sec"].map(sec_to_hms)
    df["duration_min"]= df["duration_s"] / 60.0

    csv_events = out_dir / "cow_activity_events.csv"
    df.sort_values(["cow_id","start_frame"]).to_csv(csv_events, index=False)
    print(f"Saved event log: {csv_events}")

    agg = (df.groupby(["cow_id","activity"])["duration_s"].sum().reset_index())
    agg["duration_min"] = agg["duration_s"] / 60.0
    csv_totals = out_dir / "cow_activity_totals.csv"
    agg.sort_values(["cow_id","duration_s"], ascending=[True, False]).to_csv(csv_totals, index=False)
    print(f"Saved totals:    {csv_totals}")

    print("\nSample summary:")
    view = (agg.sort_values(["cow_id","duration_s"], ascending=[True, False])
              .groupby("cow_id").head(5))
    for _, row in view.iterrows():
        print(f"cow {int(row['cow_id'])}: {row['activity']} for {row['duration_min']:.1f} min")
    return csv_events, csv_totals

csv_events, csv_totals = save_and_summarize(events, OUT_DIR, video_fps)


Saved event log: /content/cow_logs/cow_activity_events.csv
Saved totals:    /content/cow_logs/cow_activity_totals.csv

Sample summary:
cow 1: Ruminating & Standing for 2.5 min
cow 1: Ruminating & Lying for 0.4 min
cow 2: Drinking for 2.5 min
cow 2: Feeding & Lying for 0.4 min
cow 3: Drinking for 1.2 min
cow 3: Ruminating & Standing for 1.1 min
cow 3: Ruminating & Lying for 0.6 min
cow 4: Drinking for 0.8 min
cow 4: Feeding & Standing for 0.4 min
cow 4: Ruminating & Standing for 0.3 min
cow 4: Ruminating & Lying for 0.1 min
cow 5: Feeding & Standing for 0.4 min
cow 5: Ruminating & Lying for 0.2 min
cow 5: Ruminating & Standing for 0.1 min
cow 6: Ruminating & Lying for 1.5 min
cow 6: Drinking for 0.8 min
cow 6: Feeding & Lying for 0.6 min
cow 6: Ruminating & Standing for 0.1 min
cow 7: Drinking for 2.4 min
cow 7: Ruminating & Lying for 0.3 min
cow 7: Feeding & Lying for 0.2 min
cow 8: Drinking for 1.8 min
cow 8: Feeding & Lying for 0.8 min
cow 8: Ruminating & Standing for 0.2 min
cow 8: 